<a href="https://colab.research.google.com/github/FaizaAbbasi988/ShallowNet-on-EEG-Data/blob/main/ShallowNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install einops #to simplify and streamline the manipulation of tensor shapes and dimensions

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import math
import glob
import random
import itertools
import datetime
import time
import sys
import scipy.io

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchsummary import summary
import torch.autograd as autograd
from torchvision.models import vgg19

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn.init as init

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce

import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.backends import cudnn
cudnn.benchmark = False
cudnn.deterministic = True
writer = SummaryWriter('./TensorBoardX/')

In [ ]:

# Convolution module
class CNNmod(nn.Module):
    def __init__(self):
        # self.patch_size = patch_size
        super().__init__()

        self.shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), (1, 1)),
            nn.Conv2d(40, 40, (22, 1), (1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),
            nn.Dropout(0.5),
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.shallownet(x)
        return x

class ClassificationHead(nn.Module):
    def __init__(self,  n_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(2440, 256),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 32),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(32, 4)
        )

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        out = self.fc(x)
        return x, out
class CNN(nn.Sequential):
    def __init__(self, n_classes=4, **kwargs):
        super().__init__(
            CNNmod(),
            ClassificationHead(n_classes),
        )

In [18]:
root = '/content/drive/MyDrive/sortedData/BCIcompIV-2a/sorted1'

# Load training data
total_data = scipy.io.loadmat(root + 'T.mat')
train_data = total_data['data']
train_label = total_data['label']

# Transpose and expand dimensions
train_data = np.transpose(train_data, (2, 1, 0))
train_data = np.expand_dims(train_data, axis=1)
train_label = np.transpose(train_label)

# Shuffle the data
shuffle_num = np.random.permutation(len(train_data))
train_data = train_data[shuffle_num, :, :, :]
train_label = train_label[:, shuffle_num]

# Load test data
test_tmp = scipy.io.loadmat(root + 'E.mat')
test_data = test_tmp['data']
test_label = test_tmp['label']

# Transpose and expand dimensions
test_data = np.transpose(test_data, (2, 1, 0))
test_data = np.expand_dims(test_data, axis=1)
test_label = np.transpose(test_label)

# Standardize data
target_mean = np.mean(train_data)
target_std = np.std(train_data)
train_data = (train_data - target_mean) / target_std
test_data = (test_data - target_mean) / target_std

print("The shape of training data is", train_data.shape)
print("The shape of test data is", test_data.shape)
print("The shape of train label is", train_label.shape)
print("The shape of test label is", test_label.shape)
test_label= test_label-1
test_label
train_label= train_label-1


The shape of training data is (288, 1, 22, 1000)
The shape of test data is (288, 1, 22, 1000)
The shape of train label is (1, 288)
The shape of test label is (1, 288)


In [19]:
class ExP():
    def __init__(self, nsub):
        super(ExP, self).__init__()
        self.batch_size = 72
        self.n_epochs = 50
        # self.c_dim = 4
        self.lr = 0.001
        self.b1 = 0.5
        self.b2 = 0.999
        self.nSub = nsub

        self.start_epoch = 0
        self.Tensor = torch.FloatTensor
        self.LongTensor = torch.LongTensor

        self.criterion_l1 = torch.nn.L1Loss()
        self.criterion_l2 = torch.nn.MSELoss()
        self.criterion_cls = torch.nn.CrossEntropyLoss()

        self.model = CNN()

    def get_source_data(self):
        self.allData = train_data
        self.allLabel = train_label[0]
        self.testData = test_data
        self.testLabel = test_label[0]
        return self.allData, self.allLabel, self.testData, self.testLabel

    def train(self):

        img, label, test_data, test_label = self.get_source_data()

        img = torch.from_numpy(img)
        label = torch.from_numpy(label)
        dataset = torch.utils.data.TensorDataset(img, label)
        self.dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=self.batch_size, shuffle=True)

        test_data = torch.from_numpy(test_data)
        test_label = torch.from_numpy(test_label)
        test_dataset = torch.utils.data.TensorDataset(test_data, test_label)
        self.test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=self.batch_size, shuffle=True)

        # Optimizers
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, betas=(self.b1, self.b2))

        test_data = Variable(test_data.type(self.Tensor))
        test_label = Variable(test_label.type(self.LongTensor))

        bestAcc = 0
        averAcc = 0
        num = 0
        Y_true = 0
        Y_pred = 0

        # Train the cnn model
        total_step = len(self.dataloader)
        curr_lr = self.lr

        for e in range(self.n_epochs):
            # in_epoch = time.time()
            self.model.train()
            for i, (img, label) in enumerate(self.dataloader):

                img = Variable(img.type(self.Tensor))
                label = Variable(label.type(self.LongTensor))
                tok, outputs = self.model(img)

                loss = self.criterion_cls(outputs, label)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            # out_epoch = time.time()


            # test process
            if (e + 1) % 1 == 0:
                self.model.eval()
                Tok, Cls = self.model(test_data)


                loss_test = self.criterion_cls(Cls, test_label)
                y_pred = torch.max(Cls, 1)[1]
                # print("cls", Cls)
                # print("y_pred", y_pred)
                acc = float((y_pred == test_label).cpu().numpy().astype(int).sum()) / float(test_label.size(0))
                train_pred = torch.max(outputs, 1)[1]
                train_acc = float((train_pred == label).cpu().numpy().astype(int).sum()) / float(label.size(0))

                print('Epoch:', e,
                      '  Train loss: %.6f' % loss.detach().cpu().numpy(),
                      '  Test loss: %.6f' % loss_test.detach().cpu().numpy(),
                      '  Train accuracy %.6f' % train_acc,
                      '  Test accuracy is %.6f' % acc)


                num = num + 1
                averAcc = averAcc + acc
                if acc > bestAcc:
                    bestAcc = acc
                    Y_true = test_label
                    Y_pred = y_pred

        averAcc = averAcc / num
        print('The average accuracy is:', averAcc)
        print('The best accuracy is:', bestAcc)

        return bestAcc, averAcc, Y_true, Y_pred



def main():
    best = 0
    aver = 0


    for i in range(1):
        starttime = datetime.datetime.now()


        seed_n = np.random.randint(2021)
        print('seed is ' + str(seed_n))
        random.seed(seed_n)
        np.random.seed(seed_n)
        torch.manual_seed(seed_n)
        torch.manual_seed(seed_n)
        torch.cuda.manual_seed_all(seed_n)


        print('Subject %d' % (i+1))
        exp = ExP(i + 1)

        bestAcc, averAcc, Y_true, Y_pred = exp.train()
        endtime = datetime.datetime.now()
        print('subject %d duration: '%(i+1) + str(endtime - starttime))
        best = best + bestAcc
        aver = aver + averAcc
        if i == 0:
            yt = Y_true
            yp = Y_pred
        else:
            yt = torch.cat((yt, Y_true))
            yp = torch.cat((yp, Y_pred))


    best = best / 9
    aver = aver / 9



if __name__ == "__main__":
    print(time.asctime(time.localtime(time.time())))
    main()
    print(time.asctime(time.localtime(time.time())))



Fri Apr 19 10:19:44 2024
seed is 957
Subject 1
Epoch: 0   Train loss: 1.369003   Test loss: 1.384547   Train accuracy 0.305556   Test accuracy is 0.256944
Epoch: 1   Train loss: 1.361542   Test loss: 1.372963   Train accuracy 0.375000   Test accuracy is 0.250000
Epoch: 2   Train loss: 1.306024   Test loss: 1.310905   Train accuracy 0.361111   Test accuracy is 0.434028
Epoch: 3   Train loss: 1.125802   Test loss: 1.193589   Train accuracy 0.597222   Test accuracy is 0.493056
Epoch: 4   Train loss: 0.995929   Test loss: 1.028757   Train accuracy 0.500000   Test accuracy is 0.538194
Epoch: 5   Train loss: 0.862747   Test loss: 0.992401   Train accuracy 0.652778   Test accuracy is 0.496528
Epoch: 6   Train loss: 0.836226   Test loss: 0.939033   Train accuracy 0.611111   Test accuracy is 0.513889
Epoch: 7   Train loss: 0.827666   Test loss: 0.838037   Train accuracy 0.722222   Test accuracy is 0.618056
Epoch: 8   Train loss: 0.799070   Test loss: 0.940757   Train accuracy 0.625000   Test ac